In [38]:
import numpy as np
import pandas as pd
#import datetime as dt
import pymongo
import json
import datetime as dt
import pickle

In [39]:
# create mongo db connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# connect to my nba data db
db = client.nba_data_db

In [18]:
temp = db.training_data.find()
temp = list(temp)
for i in temp:
    i.pop('_id', None)

In [19]:
df = pd.DataFrame(temp)

In [21]:
len(df)

2460

In [43]:
X = df.loc[:,['eFG%', 'FTARate', 'TOV%', 'OREB%', 'OppFTARate', 'OppOREB%', 'OppTOV%', 'OppeFG%']]
y = df['W/L']

In [44]:
y.shape

(2460,)

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [46]:
X_scaler = StandardScaler().fit(X)

In [42]:
from sklearn.externals import joblib
#scaler_filename = "fourfactor_scaler.save"
#joblib.dump(X_scaler, scaler_filename)

In [28]:
ff_scaler = joblib.load(scaler_filename)

In [52]:
X = df.loc[:,['TS%', 'TOV%', 'OREB%', 'OffEff', 'DefEff', 'OppFTARate', 'OppOREB%', 'OppTOV%', 'OppeFG%']]
y = df['W/L']

In [53]:
X_scaler = StandardScaler().fit(X)

In [ ]:
ff_model = 

In [31]:
myscaler_filename = "my_scaler.save"
joblib.dump(X_scaler, myscaler_filename)

['my_scaler.save']

In [32]:
my_scaler = joblib.load(myscaler_filename)

In [35]:
fourfactormodel_filename = 'fourfactor.pkl'
fourfactor_model = pickle.load(open(fourfactormodel_filename, 'rb'))

In [36]:
mymodel_filename = 'mymodel.pkl'
my_model = pickle.load(open(mymodel_filename, 'rb'))

In [57]:
global my_model, my_scalar, ff_model, ff_scalar

NameError: name 'my_scalar' is not defined

In [56]:
ff_scaler

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'import numpy as np\nimport pandas as pd\n#import datetime as dt\nimport pymongo\nimport json',
  "# create mongo db connection\nconn = 'mongodb://localhost:27017'\nclient = pymongo.MongoClient(conn)\n# connect to my nba data db\ndb = client.nba_data_db",
  'temp = db.fourfactor_data.find({\'GameDate\': {$gt: "10/01/2018"}})\ntemp = list(temp)\nfor i in temp:\n    i.pop(\'_id\', None)',
  'db.fourfactor_data.find().forEach(function(element){\n  element.GameDate = ISODate(element.OrderDate);\n  db.collection.save(element);\n})',
  'db.fourfactor_data.find()',
  'temp = db.fourfactor_data.find()',
  'temp[0]',
  'db.fourfactor_data.find().forEach(function(element){\n  element.GameDate = ISODate(element.GameDate);\

In [50]:
def initModel():
    global my_model, my_scalar, ff_model, ff_scalar
    ffscaler_filename = "fourfactor_scaler.save"
    myscaler_filename = "my_scaler.save"
    ffmodel_filename = 'fourfactor.pkl'
    mymodel_filename = 'mymodel.pkl'
    ff_scaler = joblib.load(ffscaler_filename)
    my_scalar = joblib.load(myscaler_filename)
    ff_model = pickle.load(open(ffmodel_filename, 'rb'))
    my_model = pickle.load(open(mymodel_filename, 'rb'))

In [54]:
ffscaler_filename = "fourfactor_scaler.save"
myscaler_filename = "my_scaler.save"
ffmodel_filename = 'fourfactor.pkl'
mymodel_filename = 'mymodel.pkl'
ff_scaler = joblib.load(ffscaler_filename)
my_scalar = joblib.load(myscaler_filename)
ff_model = pickle.load(open(ffmodel_filename, 'rb'))
my_model = pickle.load(open(mymodel_filename, 'rb'))

In [52]:
ff_scalar

In [63]:
team = 'LAL'
def predictTeamRecord(team):
    global my_model, my_scalar, ff_model, ff_scalar
    db = client.nba_data_db
    temp = db.testing_data.find({'Team': team})
    temp = list(temp)
    for i in temp:
        i.pop('_id', None)
    df = pd.DataFrame(temp)
    # predict for fourfactor
    X_ff = df.loc[:,['eFG%', 'FTARate', 'TOV%', 'OREB%', 'OppFTARate', 'OppOREB%', 'OppTOV%', 'OppeFG%']]
    X_ff_scaled = ff_scaler.transform(X_ff)
    y = df['W/L']
    ff_score = ff_model.score(X_ff_scaled, y)
    ff_predictions = ff_model.predict(X_ff_scaled)
    X_my = df.loc[:,['TS%', 'TOV%', 'OREB%', 'OffEff', 'DefEff', 'OppFTARate', 'OppOREB%', 'OppTOV%', 'OppeFG%']]
    X_my_scaled = my_scaler.transform(X_my)
    my_score = my_model.score(X_my_scaled, y)
    my_predictions = my_model.predict(X_my_scaled)
    
    predict_ff_wincount = 0
    predict_my_wincount = 0
    actualwincount = 0
    for x in range (len(my_predictions)):
        if(ff_predictions[x] == 'W'):
            predict_ff_wincount += 1
        if(my_predictions[x] == 'W'):
            predict_my_wincount += 1
        if(y[x] == 'W'):
            actualwincount += 1
    results = {'FourFactor': [predict_ff_wincount, len(ff_predictions)-predict_ff_wincount, ff_score],
               'MyModel' : [predict_my_wincount, len(my_predictions)-predict_my_wincount, my_score],
               'Actual' : [actualwincount, len(y)-actualwincount, 0]
              }
    return results

temp = predictTeamRecord(team)
temp

{'FourFactor': [29, 19, 0.9166666666666666],
 'MyModel': [26, 22, 0.9375],
 'Actual': [25, 23, 0]}

In [78]:
def getTeamStats(team):
    db = client.nba_data_db
    temp = db.testing_data.find({'Team': team})
    temp = list(temp)
    for i in temp:
        i.pop('_id', None)
    df = pd.DataFrame(temp)
    df = df.groupby(['W/L'])['AST%', 'AST/TO', 'ASTRatio', 'DREB%', 
                                     'DefRtg','OREB%', 'OffRtg', 'PACE', 
                                     'PIE', 'REB%', 'TOV%','TS%',
                                     'eFG%', 'FTARate', 'OppFTARate', 'OppOREB%',
                                     'OppTOV%', 'OppeFG%'].mean()
    df = df.reset_index()
    df = df.sort_values(['W/L'])
    return df.to_json(orient='records')

In [79]:
data = getTeamStats(team)
data

'[{"W\\/L":"L","AST%":55.5086956522,"AST\\/TO":1.5582608696,"ASTRatio":16.2565217391,"DREB%":71.6260869565,"DefRtg":113.3695652174,"OREB%":24.3956521739,"OffRtg":103.3739130435,"PACE":105.0986956522,"PIE":44.5,"REB%":48.1869565217,"TOV%":14.5739130435,"TS%":53.8217391304,"eFG%":51.0913043478,"FTARate":0.2802173913,"OppFTARate":0.2665652174,"OppOREB%":28.3739130435,"OppTOV%":12.3913043478,"OppeFG%":52.5217391304},{"W\\/L":"W","AST%":58.536,"AST\\/TO":1.728,"ASTRatio":17.776,"DREB%":73.992,"DefRtg":100.416,"OREB%":28.796,"OffRtg":111.152,"PACE":102.988,"PIE":56.72,"REB%":52.68,"TOV%":15.488,"TS%":57.268,"eFG%":54.736,"FTARate":0.26124,"OppFTARate":0.21224,"OppOREB%":26.012,"OppTOV%":15.1,"OppeFG%":48.436}]'

In [72]:
db = client.nba_data_db
temp = db.nba_stats_data.find()
temp = list(temp)
for i in temp:
    i.pop('_id', None)
df = pd.DataFrame(temp)

In [75]:
df.sort_values(['W/L'])

,AST%,AST/TO,ASTRatio,DREB%,DefRtg,FTARate,OREB%,OffRtg,OppFTARate,OppOREB%,OppTOV%,OppeFG%,PACE,PIE,REB%,TOV%,TS%,W/L,eFG%
0,57.717623,1.617116,16.244703,72.465375,113.909096,0.248002,25.943204,102.500103,0.269532,27.535607,14.013075,55.206512,99.063003,43.040155,48.121240,14.768992,52.902791,L,49.344703
1,59.914367,1.958408,18.224238,74.058346,102.500103,0.269532,27.535607,113.909096,0.248002,25.943204,14.768992,49.344703,99.063003,56.960310,51.879535,14.013075,58.686202,W,55.206512


In [5]:
team1 = 'LAL'
team2 = 'WAS'
db = client.nba_data_db
temp = db.testing_data.find({'$or' : [{'Team': team1}, {'Team': team2}]})
temp = list(temp)
for i in temp:
    i.pop('_id', None)
df = pd.DataFrame(temp)

In [7]:
result = df.groupby(['Team'])['eFG%', 'FTARate', 'TOV%', 'OREB%', 'OppFTARate', 'OppOREB%', 'OppTOV%', 'OppeFG%'].mean()

In [23]:
result

,eFG%,FTARate,TOV%,OREB%,OppFTARate,OppOREB%,OppTOV%,OppeFG%
Team,,,,,,,,
LAL,53.034694,0.268102,15.138776,26.675510,0.238408,27.524490,13.795918,50.344898
WAS,52.704255,0.270404,13.646809,24.189362,0.274213,30.234043,16.495745,54.010638


In [25]:
result.loc[team1,['OppFTARate', 'OppOREB%', 'OppTOV%', 'OppeFG%']] = result.loc[team2,['FTARate', 'OREB%', 'TOV%', 'eFG%']]

In [30]:
result.loc[team1,'OppFTARate'] = result.loc[team2,'FTARate']
result.loc[team1,'OppOREB%'] = result.loc[team2,'OREB%']
result.loc[team1,'OppTOV%'] = result.loc[team2,'TOV%']
result.loc[team1,'OppeFG%'] = result.loc[team2,'eFG%']

In [31]:
result

,eFG%,FTARate,TOV%,OREB%,OppFTARate,OppOREB%,OppTOV%,OppeFG%
Team,,,,,,,,
LAL,53.034694,0.268102,15.138776,26.675510,0.270404,24.189362,13.646809,52.704255
WAS,52.704255,0.270404,13.646809,24.189362,0.274213,30.234043,16.495745,54.010638


In [11]:
result_2 = df.groupby(['Team'])['TS%', 'TOV%', 'OREB%', 'OffRtg', 'DefRtg', 'OppFTARate', 'OppOREB%', 'OppTOV%', 'OppeFG%'].mean()

In [12]:
result_2.head()

,TS%,TOV%,OREB%,OffRtg,DefRtg,OppFTARate,OppOREB%,OppTOV%,OppeFG%
Team,,,,,,,,,
LAL,55.632653,15.138776,26.675510,107.312245,106.802041,0.238408,27.524490,13.795918,50.344898
WAS,56.244681,13.646809,24.189362,108.459574,111.274468,0.274213,30.234043,16.495745,54.010638


In [17]:
temp = db.nba_stats_data.find({'W/L': 'W'})
temp = list(temp)
for i in temp:
    i.pop('_id', None)
df_stat = pd.DataFrame(temp)

In [19]:
result_2['OffEff'] = np.where(result_2['OffRtg']>=df_stat.loc[0,'OffRtg'], 1, 0)
result_2['DefEff'] = np.where(result_2['DefRtg']<=df_stat.loc[0,'DefRtg'], 1, 0)

In [32]:
result_2 = result_2.drop(['OffRtg', 'DefRtg'], axis=1)
result_2

,TS%,TOV%,OREB%,OppFTARate,OppOREB%,OppTOV%,OppeFG%,OffEff,DefEff
Team,,,,,,,,,
LAL,55.632653,15.138776,26.675510,0.238408,27.524490,13.795918,50.344898,0,0
WAS,56.244681,13.646809,24.189362,0.274213,30.234043,16.495745,54.010638,0,0


In [33]:
result_2.loc[team1,'OppFTARate'] = result.loc[team2,'FTARate']
result_2.loc[team1,'OppOREB%'] = result.loc[team2,'OREB%']
result_2.loc[team1,'OppTOV%'] = result.loc[team2,'TOV%']
result_2.loc[team1,'OppeFG%'] = result.loc[team2,'eFG%']

In [35]:
result_2 = result_2[['TS%', 'TOV%', 'OREB%', 'OffEff', 'DefEff', 'OppFTARate', 'OppOREB%', 'OppTOV%', 'OppeFG%']]
result_2

,TS%,TOV%,OREB%,OffEff,DefEff,OppFTARate,OppOREB%,OppTOV%,OppeFG%
Team,,,,,,,,,
LAL,55.632653,15.138776,26.675510,0,0,0.270404,24.189362,13.646809,52.704255
WAS,56.244681,13.646809,24.189362,0,0,0.274213,30.234043,16.495745,54.010638


In [56]:
X_ff = result.loc[:,['eFG%', 'FTARate', 'TOV%', 'OREB%', 'OppFTARate', 'OppOREB%', 'OppTOV%', 'OppeFG%']]

In [57]:
X_ff_scaled = ff_scaler.transform(X_ff)

In [58]:
ff_predictions = ff_model.predict(X_ff_scaled)

In [73]:
ff_predictions[0]

'W'

In [70]:
temp = result.index[0]

In [72]:
if(temp == team1):
    print("Match")
else:
    print("No Match")

Match


In [69]:
result

,eFG%,FTARate,TOV%,OREB%,OppFTARate,OppOREB%,OppTOV%,OppeFG%
Team,,,,,,,,
LAL,53.034694,0.268102,15.138776,26.675510,0.270404,24.189362,13.646809,52.704255
WAS,52.704255,0.270404,13.646809,24.189362,0.274213,30.234043,16.495745,54.010638
